In [5]:
import pyspark
from pyspark import SparkContext
sc = SparkContext('local[8]')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[8]) created by __init__ at <ipython-input-2-ff81a32269f4>:3 

In [6]:
url = "pokemon.csv"
from pyspark import SparkFiles
from pyspark.sql import Row
from pyspark.sql import SQLContext
sc.addFile(url)
sqlContext = SQLContext(sc)

In [8]:
df = sqlContext.read.csv(SparkFiles.get("pokemon.csv"), header=True, inferSchema= True)

In [9]:
df.printSchema()

root
 |-- abilities: string (nullable = true)
 |-- against_bug: double (nullable = true)
 |-- against_dark: double (nullable = true)
 |-- against_dragon: double (nullable = true)
 |-- against_electric: double (nullable = true)
 |-- against_fairy: double (nullable = true)
 |-- against_fight: double (nullable = true)
 |-- against_fire: double (nullable = true)
 |-- against_flying: double (nullable = true)
 |-- against_ghost: double (nullable = true)
 |-- against_grass: double (nullable = true)
 |-- against_ground: double (nullable = true)
 |-- against_ice: double (nullable = true)
 |-- against_normal: double (nullable = true)
 |-- against_poison: double (nullable = true)
 |-- against_psychic: double (nullable = true)
 |-- against_rock: double (nullable = true)
 |-- against_steel: double (nullable = true)
 |-- against_water: double (nullable = true)
 |-- attack: integer (nullable = true)
 |-- base_egg_steps: integer (nullable = true)
 |-- base_happiness: integer (nullable = true)
 |-- bas

In [10]:
df.show(5)

+--------------------+-----------+------------+--------------+----------------+-------------+-------------+------------+--------------+-------------+-------------+--------------+-----------+--------------+--------------+---------------+------------+-------------+-------------+------+--------------+--------------+----------+------------+--------------+-------+-----------------+--------+---+---------------------+----------+---------------+--------------+---------+----------+-----+-----+------+---------+----------+------------+
|           abilities|against_bug|against_dark|against_dragon|against_electric|against_fairy|against_fight|against_fire|against_flying|against_ghost|against_grass|against_ground|against_ice|against_normal|against_poison|against_psychic|against_rock|against_steel|against_water|attack|base_egg_steps|base_happiness|base_total|capture_rate| classfication|defense|experience_growth|height_m| hp|        japanese_name|      name|percentage_male|pokedex_number|sp_attack|sp_

In [12]:
df.count()

801

In [16]:
df.select('attack', 'defense').show(5)

+------+-------+
|attack|defense|
+------+-------+
|    49|     49|
|    62|     63|
|   100|    123|
|    52|     43|
|    64|     58|
+------+-------+
only showing top 5 rows



In [25]:
# How many are there of each type?
df.groupBy("type1").count().sort("count", ascending=False).show()

+--------+-----+
|   type1|count|
+--------+-----+
|   water|  114|
|  normal|  105|
|   grass|   78|
|     bug|   72|
| psychic|   53|
|    fire|   52|
|    rock|   45|
|electric|   39|
|  ground|   32|
|  poison|   32|
|    dark|   29|
|fighting|   28|
|  dragon|   27|
|   ghost|   27|
|   steel|   24|
|     ice|   23|
|   fairy|   18|
|  flying|    3|
+--------+-----+



In [22]:
df.describe('attack').show()

+-------+-----------------+
|summary|           attack|
+-------+-----------------+
|  count|              801|
|   mean|77.85767790262172|
| stddev|32.15882023801748|
|    min|                5|
|    max|              185|
+-------+-----------------+



In [35]:
# Are there more or less types of each each generation
df.crosstab('type1', 'generation').sort('type1_generation').show()

+----------------+---+---+---+---+---+---+---+
|type1_generation|  1|  2|  3|  4|  5|  6|  7|
+----------------+---+---+---+---+---+---+---+
|             bug| 12| 10| 12|  8| 18|  3|  9|
|            dark|  0|  5|  4|  3| 13|  3|  1|
|          dragon|  3|  0|  7|  3|  7|  4|  3|
|        electric|  9|  6|  4|  7|  7|  3|  3|
|           fairy|  2|  5|  0|  1|  0|  9|  1|
|        fighting|  7|  2|  4|  2|  7|  3|  3|
|            fire| 12|  8|  6|  5|  8|  8|  5|
|          flying|  0|  0|  0|  0|  1|  2|  0|
|           ghost|  3|  1|  4|  6|  5|  4|  4|
|           grass| 12|  9| 12| 13| 15|  5| 12|
|          ground|  8|  3|  6|  4|  9|  0|  2|
|             ice|  2|  4|  6|  3|  6|  2|  0|
|          normal| 22| 15| 18| 17| 17|  4| 12|
|          poison| 14|  1|  3|  6|  2|  2|  4|
|         psychic|  8|  7|  8|  7| 14|  3|  6|
|            rock|  9|  4|  8|  6|  6|  8|  4|
|           steel|  0|  2|  9|  3|  4|  4|  2|
|           water| 28| 18| 24| 13| 17|  5|  9|
+------------

In [38]:
df.filter(df.attack > 100).count()

169

In [41]:
df.groupby('type1').agg({'attack': 'mean'}).sort('avg(attack)', ascending=False).show()

+--------+------------------+
|   type1|       avg(attack)|
+--------+------------------+
|  dragon| 106.4074074074074|
|fighting| 99.17857142857143|
|  ground|           94.8125|
|   steel| 93.08333333333333|
|    rock| 90.66666666666667|
|    dark| 87.79310344827586|
|    fire|              81.5|
|  normal| 75.16190476190476|
|   grass| 73.76923076923077|
|   water| 73.30701754385964|
|     ice| 73.30434782608695|
|   ghost| 72.74074074074075|
|  poison|          72.65625|
|electric| 70.82051282051282|
|     bug|            70.125|
|  flying| 66.66666666666667|
| psychic| 65.56603773584905|
|   fairy|62.111111111111114|
+--------+------------------+



In [57]:
from pyspark.sql.functions import *
# 1 Select the column
#full_attack = df.select(col("attack") + col('sp_attack'))
# 2 Apply the transformation and add it to the DataFrame
df = df.withColumn("full_attack", col("attack") + col('sp_attack'))

In [58]:
df.select('attack', 'sp_attack', 'full_attack').show(5)

+------+---------+-----------+
|attack|sp_attack|full_attack|
+------+---------+-----------+
|    49|       65|        114|
|    62|       80|        142|
|   100|      122|        222|
|    52|       60|        112|
|    64|       80|        144|
+------+---------+-----------+
only showing top 5 rows

